In [ ]:
픽셀을 28 x 28일 때 성능을 검증하기 위한 코드입니다.
원래는 정의해 놓은 함수를 불러와서 사용해야 했으나, 검증 과정에서 일부 함수를 직접 코드를 복사해와서 테스트 한 코드도 많습니다.

In [1]:
import cv2
from PIL import Image
import sys
import os
import numpy as np

current_dir = os.getcwd()
sys.path.append(current_dir)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
image_folder_path = './G1020/Images/'    

In [3]:
import pandas as pd
df = pd.read_csv('G1020.csv')
image_files = df['imageID'].tolist()  # 이미지 파일 이름이 있는 열 이름
labels = df['binaryLabels'].values  # 레이블이 있는 열 이름


# 이미지 크기 설정
target_size = (28, 28)

# 이미지 불러오기 및 전처리 함수
def load_and_preprocess_image(image_path, target_size=(28, 28)):
    # 이미지 읽기 (기본 BGR 형식)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 흑백으로 읽기
    if image is None:
        print(f"Image not found: {image_path}")
        return None
    
    # 이미지 크기 조정 (28x28)
    image = cv2.resize(image, target_size)
    
    # 정규화 (0~1 범위로 스케일링)
    image = image / 255.0
    
    # 차원을 추가하여 (1, 28, 28)로 만들기
    image = np.expand_dims(image, axis=0)
    
    return image

# 모든 이미지를 불러와서 리스트에 저장
images = [load_and_preprocess_image(image_folder_path+img_path, target_size) for img_path in image_files]
images = np.array([img for img in images if img is not None])  # None 값 제거

print(f"Processed images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Processed images shape: (1020, 1, 28, 28)
Labels shape: (1020,)


In [39]:
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int((input_size - filter_size + 2 * filter_pad) / filter_stride + 1)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))


        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b3'] = np.zeros(hidden_size)        
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b4'] = np.zeros(hidden_size)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b5'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.layers['Relu3'] = Relu()  # 추가된 ReLU 활성화 함수
        self.layers['Affine3'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Relu4'] = Relu()  # 추가된 ReLU 활성화 함
        self.layers['Affine4'] = Affine(self.params['W5'], self.params['b5'])

        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
        # for idx in (1, 2, 3,4,5):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W4'], grads['b4'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        grads['W5'], grads['b5'] = self.layers['Affine4'].dW, self.layers['Affine4'].db

        return grads

In [40]:
from common.trainer import Trainer
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.300803272950098
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.298882390785243
train loss:2.2967263456152085
train loss:2.2940187194229975
train loss:2.290784882541292
train loss:2.2864484700638927
train loss:2.2796501963985305
train loss:2.265035361444203
train loss:2.247697226529353
train loss:2.2165315608839435
train loss:2.1780456498026424
train loss:2.115740964265553
train loss:2.0314443772859767
train loss:1.9299488396764055
train loss:1.6407425800398499
train loss:1.389671495126504
train loss:1.2421294987329223
train loss:1.0254760422462321
train loss:0.7142180101166379
train loss:0.8975504013894401
train loss:0.6193962273020703
train loss:1.165636264280297
train loss:0.7362177617594363
train loss:0.7033095832589078
train loss:0.5160771383905955
train loss:0.6791765825737472
train loss:0.685520622300128
train loss:0.6575167026355455
train loss:0.6782074385431407
train loss:0.6701157715408161
train loss:0.525056065088166
train loss:0.6512592873486155
trai

In [2]:
import pandas as pd
df = pd.read_csv('G1020.csv')
image_files = df['imageID'].tolist()  # 이미지 파일 이름이 있는 열 이름
labels = df['binaryLabels'].values  # 레이블이 있는 열 이름

# 이미지 크기 설정
target_size = (28, 28)

# cv2로 이미지 읽고 전처리
def load_and_extract_hue(image_path, target_size):
    image = cv2.imread(image_path)  # 이미지 읽기 (기본 BGR 형식)
    if image is None:
        print(f"Image not found: {image_path}")
        return None
    image = cv2.resize(image, target_size)  # 이미지 크기 조정
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # BGR을 HSV로 변환
    hue_channel = hsv_image[:, :, 0]  # Hue 채널만 추출
    hue_channel = hue_channel / 180.0  # Hue 값 정규화 (0~1 범위로 스케일링, Hue 범위는 0-179)
    return hue_channel

# 모든 이미지를 불러와서 리스트에 저장
hue_images = [load_and_extract_hue(image_folder_path + img_path, target_size) for img_path in image_files]
hue_images = np.array([img for img in hue_images if img is not None])  # None 값 제거

# 차원 추가하여 (1020, 1, 128, 128) 형태로 변환
hue_images = hue_images[:, np.newaxis, :, :]

print(f"Processed Hue images shape: {hue_images.shape}")
print(f"Labels shape: {labels.shape}")


Processed Hue images shape: (1020, 1, 28, 28)
Labels shape: (1020,)


In [7]:
from common.trainer import Trainer

x_train, x_test, y_train, y_test = train_test_split(hue_images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.2998602638334167
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.295370330187135
train loss:2.2888145706905396
train loss:2.2778996305229873
train loss:2.2555070292030237
train loss:2.234920183588287
train loss:2.1947911513570335
train loss:2.1551512257486967
train loss:2.0552867723339387
train loss:2.0179849263991274
train loss:1.9067493226918835
train loss:1.785065347923328
train loss:1.5873058487475011
train loss:1.4581537769916273
train loss:1.2636267089883366
train loss:1.2251661951109598
train loss:0.7659249284554346
train loss:0.712054079773794
train loss:0.7691612056074145
train loss:0.5934447851069408
train loss:0.7152877040142341
train loss:0.9690796665957601
train loss:0.8859399633192634
train loss:0.5373714122178896
train loss:0.8300782836841927
train loss:0.5238847499384778
train loss:0.6103711892762114
train loss:0.7637493489532398
train loss:0.6221800310141692
train loss:0.5273797812913722
train loss:0.5680836707586787
train loss:0.628433657288921

In [8]:
class DeepConvNet:
    def __init__(self, input_dim=(1, 28, 28), #필터 수 조절해서 성능 조절
                 conv_param_1 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [9]:
network = DeepConvNet()  
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

train loss:2.236009654231396
=== epoch:1, train acc:0.18872549019607843, test acc:0.2549019607843137 ===
train loss:2.157605452988102
train loss:2.105294772900327
train loss:1.9779932031732241
train loss:1.819595078525812
train loss:1.7723165942125378
train loss:1.7456789644232333
train loss:1.7176090466960974
train loss:1.7339974282477428
train loss:1.763286660694815
train loss:1.988294663312246
train loss:1.748271531562744
train loss:1.6627061434625923
train loss:1.7506131837112548
train loss:1.7559101046598244
train loss:1.6741601653035099
train loss:1.5533335726147501
train loss:1.640819854708955
train loss:1.6888797970753684
train loss:1.5070705932287052
train loss:1.6788526770864554
train loss:1.645980652414771
train loss:1.8150505464965243
train loss:1.5539968440357947
train loss:1.6604793538807112
train loss:1.7130409841886836
train loss:1.6052917978367676
train loss:1.6539645326258097
train loss:1.4561149186482565
train loss:1.6180875582276844
train loss:1.6083518950258664
tra

In [24]:
from mymethod.functions import *


x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)


model = LogisticRegression()
model.fit(x_train_reshaped, y_train)

y_pred = model.predict(x_test_reshaped)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import make_classification

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.5196078431372549
혼동 행렬:
 [[83 55]
 [43 23]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.66      0.60      0.63       138
           1       0.29      0.35      0.32        66

    accuracy                           0.52       204
   macro avg       0.48      0.47      0.47       204
weighted avg       0.54      0.52      0.53       204



In [25]:
lda = LinearDiscriminantAnalysis()
lda.fit(x_train_reshaped , y_train)
y_pred = lda.predict(x_test_reshaped)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.5490196078431373
혼동 행렬:
 [[88 50]
 [42 24]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.68      0.64      0.66       138
           1       0.32      0.36      0.34        66

    accuracy                           0.55       204
   macro avg       0.50      0.50      0.50       204
weighted avg       0.56      0.55      0.56       204



In [26]:
nb = GaussianNB()
nb.fit(x_train_reshaped , y_train)
y_pred = nb.predict(x_test_reshaped)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)


정확도: 0.4117647058823529
혼동 행렬:
 [[ 27 111]
 [  9  57]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.75      0.20      0.31       138
           1       0.34      0.86      0.49        66

    accuracy                           0.41       204
   macro avg       0.54      0.53      0.40       204
weighted avg       0.62      0.41      0.37       204



In [3]:
images = hue_images

In [24]:
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

class ConvNet3Layer:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int((input_size - filter_size + 2 * filter_pad) / filter_stride + 1)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))


        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b3'] = np.zeros(hidden_size)        
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b4'] = np.zeros(hidden_size)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b5'] = np.zeros(output_size)
        
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()

        self.layers['Conv2'] = Convolution(self.params['W3'], self.params['b3'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu2'] = Relu()
        self.layers['Pool2'] = Pooling(pool_h=2, pool_w=2, stride=2)

        self.layers['Affine2'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Relu3'] = Relu()  # 추가된 ReLU 활성화 함수
        self.layers['Affine3'] = Affine(self.params['W5'], self.params['b5'])
        
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3,4,5):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Conv2'].dW, self.layers['Conv2'].db
        grads['W4'], grads['b4'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W5'], grads['b5'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        


        return grads


In [25]:
from common.trainer import Trainer
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param1 = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
class DeepConvNet:
    def __init__(self, input_dim=(1, 28, 28), #필터 수 조절해서 성능 조절
                 conv_param_1 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]            